In [1]:
from flask import Flask, render_template, request
from flask import send_file, url_for
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os
import numpy as np

In [2]:
from bs4 import BeautifulSoup
import urllib.request
import re
from krwordrank.sentence import summarize_with_sentences

In [3]:
from selenium import webdriver
from selenium.common import exceptions

In [4]:
app = Flask(__name__)

# 크롤링

In [5]:
OUTPUT_FILE_NAME = 'output.txt'

In [6]:
wd = "C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver"

In [7]:
def get_text(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = ''
    for item in soup.find_all('div', id='articeBody'):
        text = text + str(item.find_all(text=True))
    return text

In [8]:
def clean_text(text):
    cleaned_text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\.]+", " ", text)
    return cleaned_text

In [9]:
def comment(url):
    driver = webdriver.Chrome(wd)
    driver.get(url)
    pages = 0
    try:
        while True:
            driver.find_element_by_css_selector(".u_cbox_btn_more").click()
            time.sleep(1.5)
            print(pages, end=" ")
            pages+=1
    except exceptions.ElementNotVisibleException as e: # 페이지 끝
        pass
    
    except Exception as e: # 다른 예외 발생시 확인
        print(e)
    html = driver.page_source
    dom = BeautifulSoup(html, "lxml")
    # 댓글이 들어있는 페이지 전체 크롤링
    comments_raw = dom.find_all("span", {"class" : "u_cbox_contents"})
    # 댓글의 text만 뽑는다.
    comments = [comment.text for comment in comments_raw]
    return comments

# 키워드 추출

In [10]:
def keyword_word(text):#키워드 단어 추출
    data = text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=10, num_keysents=10)
    return keywords

In [11]:
def keyword_sentence(url):#키워드 문장 추출
    m_text = get_text(url)
    r_text = clean_text(m_text)
    data = r_text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=10, num_keysents=10)
    return sents

# 플라스크

In [12]:
class ArticleForm(Form):
    articleurl = TextAreaField('',[validators.DataRequired()])

In [13]:
@app.route('/',methods = ['POST'])
def index():
    form = Reviewform(request.form)
    return render_template('article_app.html', form=form)

In [14]:
@app.route('/article',methods = ['POST'])
def article():
    form = ArticleForm(request.form)
    if request.method == 'POST' and form.validate():
        url = request.form['articleurl']
        m_text = get_text(url)
        r_text = clean_text(m_text)
        return render_template('title.html',content=r_text)
    return render_template('article_app.html',form=form)

In [15]:
@app.route('/results', methods = ['POST'])
def review():
    form = ArticleForm(request.form)
    if request.method == 'POST' and form.validate():
        url = request.form['articleurl']
        key_sentence = keyword_sentence(url)
        cr_comment = comment(url)
        return render_template('result.html',list=key_sentence, list2=cr_comment)
    return render_template('title.html',form=form)

In [16]:
@app.route('/thanks', methods = ['POST'])
def feedback():
    feedback = request.form['feedback_button']
    return render_template('thanks.html')

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Feb/2020 09:17:07] "GET / HTTP/1.1" 405 -
127.0.0.1 - - [12/Feb/2020 09:17:07] "GET /favicon.ico HTTP/1.1" 404 -
